## Trade with ChatGPT

In [2]:
import sys
sys.path.append("D:\python_project\FinRL-Meta\FinRL-Meta")  # Your FinRL-Meta dir
import pandas as pd
from meta.data_processors.akshare import Akshare
import datetime
import matplotlib.pyplot as plt

#### Read data

In [3]:
df = pd.read_csv("./data/maotai.csv")
df.head(1)

CODE  NAME                 DATE         CREATED_DATE  \
0  600519  贵州茅台  2021-01-07 18:24:00  2021-01-07 18:23:00   

                                              text_a  \
0  贵州茅台公告，董事会于2021年1月6日收到公司董事王焱的书面辞职报告，王焱因工作变动，申请...   

                                         DESCRIPTION       READ  MARKET  \
0  贵州茅台公告，董事会于2021年1月6日收到公司董事王焱的书面辞职报告，王焱因工作变动，申请...  5034011.0     1.0   

     CHANGE   FIRST_DAY  ...  \
0  0.000935  2021-01-07  ...   

                                             A0_mask  \
0  [[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....   

                                             A1_mask  AV_num  trade_date  \
0  [[0. 0. 0. ... 0. 0. 0.]\n [0. 0. 0. ... 0. 0....     3.0  2021-01-07   

  open_close_chg  co_label  cc_label  \
0       0.247727       4.0       4.0   

                                       stock_factors  label  \
0  [39.43367824665831, 39.12671101868472, 39.9102...      1   

                                         chatgpt_res  
0  公司高管变动通常会对公司的经营和股票价格产生一定的影响，但具体影响的程度和方向需要综合考虑多...  

[1 rows x 42 columns]

#### Generate Signal from ChatGPT

In [4]:
def generate_signal_from_chatgpt(x):
    chatgpt_res = x.chatgpt_res
    if "持有不动" in chatgpt_res:
        signal = 0
    if "小幅加仓" in chatgpt_res:
        signal = 1
    elif "小幅减仓" in chatgpt_res:
        signal = -1
    if "大幅加仓" in chatgpt_res:
        signal = 2
    elif "大幅减仓" in chatgpt_res:
        signal = -2
    return signal

In [5]:
df["signal"] = df.apply(generate_signal_from_chatgpt,axis = 1)

In [6]:
df["signal"]

0    -1
1     0
2     1
3     0
4     0
5     0
6     1
7     0
8     0
9     0
10   -1
11    1
12   -1
13    0
14    1
Name: signal, dtype: int64

#### Generate Signal by yourself
Here the result of ChatGPT is just a suggestion, you need to make trading signal after considering both news and the result of ChatGPT

In [7]:
def generate_signal_from_chatgpt_by_yourself(x):
    chatgpt_res = x.chatgpt_res
    news = x.text_a
    signal = input(
        f'- {news}\n - {chatgpt_res}\n (Please input 1 for buy or 0 for hold or -1 for sell)'
        )
    return int(signal)

In [8]:
df["signal_by_yourself"] = df.apply(generate_signal_from_chatgpt_by_yourself,axis = 1)

In [9]:
df["signal_by_yourself"]

0    -1
1     1
2     1
3     0
4     1
5     0
6     1
7     1
8     1
9     1
10   -1
11    1
12    1
13    0
14    1
Name: signal_by_yourself, dtype: int64

#### Fetching price data from FinRL-Meta

In [10]:
df.DATE = pd.to_datetime(df.DATE)
df.DATE = df.DATE.dt.date.astype(str)
max_date = datetime.datetime.strptime(df.DATE.max(),"%Y-%m-%d")
max_date += datetime.timedelta(days=10)
max_date = max_date.strftime("%Y-%m-%d")

In [11]:
time_interval = "daily"
start_date= df.DATE.min()
end_date = max_date
adjust=""

ticket_list=[f'{df.iloc[0].CODE}.SH', ]

In [12]:
as_processor = Akshare("akshare",start_date=start_date,end_date=end_date,time_interval=time_interval)
as_processor.download_data(ticket_list)
as_processor.dataframe.shape

100%|██████████| 1/1 [00:00<00:00,  3.00it/s]

Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (196, 8)


(196, 8)

#### Trading
Since we only have the news for several days here, we only calculated the dicision results made by Chatgpt or by combining Chatgpt and your own opinion

In [13]:
price_df = as_processor.dataframe
price_df = price_df.reset_index(drop=True)
price_df.head(1)

tic        time    open    high     low   close  volume  day
0  600519.SH  2021-01-07  2097.0  2140.0  2075.0  2140.0   37931    3

In [14]:
def cal_reward(x,by_yourself = False, hold = False):
    # get price
    this_date = x.DATE
    this_price = price_df[price_df.time == this_date]
    
    while this_price.shape[0] == 0:
        this_date = datetime.datetime.strptime(this_date,"%Y-%m-%d")
        this_date += datetime.timedelta(days=1)
        this_date = this_date.strftime("%Y-%m-%d")
        this_price = price_df[price_df.time == this_date]
    
    next_price = price_df.iloc[this_price.index+1]
    this_close = this_price.close.item()
    next_open = next_price.open.item()
    next_close = next_price.close.item() # T+1
    
    if hold:
        signal = 1
    else:
        if by_yourself:
            signal = x.signal_by_yourself
        else:
            signal = x.signal

    reward = signal * (next_close-this_close)/this_close
    return reward


In [15]:
df["reward"] = df.apply(lambda x:cal_reward(x),axis = 1)
df["reward_by_yourself"] = df.apply(lambda x:cal_reward(x,by_yourself = True),axis = 1)
df["reward_hold"] = df.apply(lambda x:cal_reward(x,hold = True),axis = 1)

In [16]:
reward_list = (df["reward"] +1 ).cumprod().to_list()
reward_hold_list = (df["reward_hold"] +1 ).cumprod().to_list()
reward_yourself_list = (df["reward_by_yourself"] +1 ).cumprod().to_list()
reward_list = [i/reward_list[0] for i in reward_list]
reward_hold_list = [i/reward_hold_list[0] for i in reward_hold_list]
reward_yourself_list = [i/reward_yourself_list[0] for i in reward_yourself_list]


In [17]:
date_list = df.DATE
plt.figure(figsize=(20,5))
plt.plot(date_list, reward_list, label = "Reward by ChatGPT")
plt.plot(date_list, reward_yourself_list, label= "Reward with ChatGPT")
plt.plot(date_list, reward_hold_list, label= "Buy and hold")
plt.title("Maotai ChatGPT Trading Results")
plt.legend()
plt.show()